In [1]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [22]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Regresión lineal

**Regresión:** Consiste en **ajustar** un modelo paramétrico
$$
f_\theta: x \rightarrow y
$$
que sea capaz de predecir $y$ dado $x$

- $x$ variable indepediente, entrada, característica, predictor
- $y$ variable dependiente, salida, respuesta, objetivo (target)
- $x$ e $y$ son variables continuas
- $\theta$ son los parámetros del modelo

> Encontrar como dos o más variables se relacionan

> Explicar/Predecir una variable en función de otras

Hablamos de **regresión lineal** cuando el modelo $f_\theta$ es **lineal en sus parámetros**

- **Datos:** conjunto de $M$ tuplas $(\vec x_i, y_i)$ con $i=1,2,\ldots,M$ 
- **Ajuste:** Encontrar el valor óptimo de $\theta$ en función de los datos
- Si  $f_\theta$ es lineal en sus parámetros se puede escribir como un **sistema lineal de $M$ ecuaciones**
- Si el sistema es rectangular lo podemos resolver con **Mínimos Cuadrados**
$$
\min_\theta \sum_{i=1}^M \left(y_i - f_\theta(\vec x_i) \right)^2
$$
- En dicho caso la solución es óptima *"en el sentido de mínimos cuadrádos"*

### Modelos lineales en sus parámetros y en sus entradas
#### Recta
Si $x$ es unidimensional 
$$
y =\theta_0  + \theta_1 x 
$$

#### Plano
Si $\vec x =(x_1, x_2)$ es bidimensional 
$$
y = \theta_0  + \theta_1 x_1 +  \theta_2 x_2 
$$

#### Hiperplano
Si $\vec x = (x_1, x_2, \ldots, x_d)$ es d-dimensional
$$
y = \theta_0  + \sum_{k=1}^d \theta_k x_k 
$$

In [12]:
from ipywidgets import interact, SelectionSlider, IntSlider
from mpl_toolkits.mplot3d import Axes3D
plt.close('all'); fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')
theta = [4, 3, 2]; 

def update(rseed, N, sigma):
    ax.cla();
    np.random.seed(rseed);
    x1, x2 = np.random.randn(2, N)
    y_clean = theta[0] + theta[1]*x1 + theta[2]*x2 
    y = y_clean + sigma*np.random.randn(len(x1))
    X_lstsq = np.stack((np.ones_like(x1), x1, x2)).T
    param, MSE, rank, singval = np.linalg.lstsq(X_lstsq, y, rcond=None)
    display(theta, param)
    ax.scatter(x1, x2, y, s=10, label='data')
    X1, X2 = np.meshgrid(np.linspace(np.amin(x1), np.amax(x1), num=2), 
                         np.linspace(np.amin(x2), np.amax(x2), num=2))
    ax.plot_surface(X1, X2, param[0] + param[1]*X1 + param[2]*X2, 
                    label='model', alpha=0.25)

interact(update, 
         rseed=IntSlider_nice(continuous_update=False), 
         N=SelSlider_nice(options=[10, 100, 1000]),
         sigma=SelSlider_nice(options=[0.1, 0.5, 1, 2, 5, 10.]));

[4, 3, 2]

array([4.01915044, 2.99668368, 2.00570852])

### Predicción: Interpolación y Extrapolación

Una vez que el regresor ha sido ajustado se puede usar para hacer predicciónes de la variable dependiente a partir de valores no observados de la variable independiente

- Llamamos interpolación cuando predecimos dentro del rango de nuestros datos
- Llamamos extrapolación cuando predecimos fuera del rango de nuestros datos

In [5]:
x = np.linspace(-1, 1, num=5)
y = lambda x: -0.75*x**2 + 5*x -4
theta = np.linalg.lstsq(np.stack((np.ones_like(x), x)).T, y(x), rcond=None)[0]
y_hat = lambda x : np.dot(np.stack((np.ones_like(x), x)).T, theta)
fig, ax = plt.subplots(figsize=(6, 3), tight_layout=True)
ax.scatter(x, y(x), c='k', label='datos observados')
ax.scatter(3, y(3), c='r', label='dato nuevo')
x_plot = np.linspace(np.amin(x), np.amax(x))
ax.plot(x_plot, y_hat(x_plot), 'k-', label='interpolación')
x_plot = np.linspace(np.amax(x), 3)
ax.plot(x_plot, y_hat(x_plot), 'k--', label='extrapolación')
ax.set_xlabel('x')
ax.set_ylabel('y');
plt.legend();

<IPython.core.display.Javascript object>

### Modelos lineales en sus parámetros pero no en sus entradas

Podemos generalizar la regresión lineal usando funciónes base $\phi_j(\cdot)$ tal que el modelo

$$
y = f_\theta (x) = \sum_{j=0}^N \theta_j \phi_j (x)
$$

#### Regresión lineal con polinomios

Si usamos $\phi_j(x) = x^j$ nos queda

$$
y = f_\theta (x) = \theta_0 + \theta_1 x + \theta_2 x^2 + \ldots
$$

#### Regresión lineal con sinusoides

Si usamos $\phi_j(x) = \cos(2\pi j x)$ nos queda

$$
y = f_\theta (x) = \theta_0 + \theta_1 \cos(2\pi x) + \theta_2 \cos(4 \pi x) + \ldots
$$

In [14]:
x = np.linspace(0, 2, num=100)
y = 2*np.cos(2.0*np.pi*x) + np.sin(4.0*np.pi*x) + 0.4*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax.scatter(x, y)

poly_basis = lambda x,N : np.vstack([x**k for k in range(N)]).T
N = 1
theta = np.linalg.lstsq(poly_basis(x, N), y, rcond=None)[0]
ax.plot(x, np.dot(poly_basis(x, N), theta));

<IPython.core.display.Javascript object>

### Reconocer un sistema lineal en sus parámetros

Un sistema lineal en sus parámetros admite una factorización
$$
f_\theta(\vec x) = \begin{pmatrix} \phi_0(\vec x) & \phi_1(\vec x) & \ldots & \phi_N(\vec x) \end{pmatrix} \begin{pmatrix} \theta_0 \\ \theta_1 \\ \vdots \\ \theta_N  \end{pmatrix}
$$

donde $\phi_j$ son transformaciones arbitrarias de la entrada


¿Son estos modelos lineales en sus parámetros?
$$
\begin{align}
y &= f_\theta(x) = \theta_0  + \theta_1 x  \nonumber \\
y &= f_\theta(x) = \theta_0  + \theta_1 x + \theta_2 x^2 + \theta_3 \log(x) \nonumber \\
y &= f_\theta(x) = \theta_0  + \sin(\theta_1) x  \nonumber 
\end{align}
$$


### Sistema infradeterminado

Es aquel sistema que tiene más incognitas (parámetros) que ecuaciones, $N>M$

Este tipo de sistema tiene infinitas soluciones

#### Ejemplo: Dos puntos con polinomio de segundo orden (tres parámetros)

In [16]:
x = np.array([-2, 2])
y = np.array([4, 4])
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
x_plot = np.linspace(-3, 3, num=100)
thetas = np.zeros(shape=(200, 3))
for i, a in enumerate(np.linspace(-10, 10, num=thetas.shape[0])):
    ax.plot(x_plot, a  + (1 - a/4)*x_plot**2)
    thetas[i:] = [a, 0, (1-a/4)]
ax.scatter(x, y, s=100, c='k', zorder=10);

<IPython.core.display.Javascript object>

En este caso $A^T A$ no es invertible 

In [17]:
A = poly_basis(x, N=3)
display(A)
np.linalg.inv(np.dot(A.T, A))

array([[ 1, -2,  4],
       [ 1,  2,  4]])

LinAlgError: Singular matrix

El problema infradeterminado se resuelve imponiendo una restricción adicional

La más típica es que el vector solución tenga norma mínima

$$
\min_\theta \| x \|_2^2 ~\text{s.a.}~ Ax =b
$$

que se resuelve usando $M$ multiplicadores de Lagrande

$$
\begin{align}
\frac{d}{dx} \| x\|_2^2 + \lambda^T (b - Ax) &= 2x - \lambda^T A  \nonumber \\
&= 2Ax - A A^T \lambda \nonumber \\
&= 2b - A A^T \lambda = 0 \nonumber \\
&\rightarrow \lambda = 2(AA^T)^{-1}b \nonumber \\
&\rightarrow x = \frac{1}{2} A^T \lambda = A^T (A A^T)^{-1} b
\end{align}
$$

donde $A^T (A A^T)^{-1}$ se conoce como la pseudo-inversa "por la derecha"

In [ ]:
x = np.array([-2, 2])
y = np.array([4, 4])
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
x_plot = np.linspace(-3, 3, num=100)
theta = np.dot(np.dot(A.T, np.linalg.inv(np.dot(A, A.T))), y)
ax.plot(x_plot, np.dot(poly_basis(x_plot, N=3), theta))
ax.scatter(x, y, s=100, c='k', zorder=10)
display(theta)
display(thetas[np.argmin(np.sum(thetas**2, axis=1)), :])

In [ ]:
np.linalg.lstsq(A, y, rcond=None)[0]

Investigando nos damos cuenta que `linalg.lstsq` está basado en la función de LAPACK [`dgels`](https://www.math.utah.edu/software/lapack/lapack-d/dgels.html)

dgels usa la pseudo inversa izquierda si $N<M$ o la pseudo inversa derecha si $N>M$

> Se asume que la mejor solución del sistema infradeterminado es la de **mínima norma euclidiana**


# Modelo con error cero

Ajustamos nuestro regresor lineal mínimizando el **la suma de errores cuadráticos**

¿Buscamos siempre un modelo de mínimo error?

In [ ]:
x = np.linspace(-3, 3, num=10)
x_plot = np.linspace(-3, 3, num=10*len(x))
y_clean = np.poly1d([2, -4, 20]) # 2*x**2 -4*x +20
y = y_clean(x) + 3*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=(5, 4), tight_layout=True)
def update_plot(N_params):
    ax.cla()
    ax.scatter(x, y); 
    ax.plot(x_plot, y_clean(x_plot), lw=2, alpha=.5)
    ax.set_xlabel('x'); ax.set_ylabel('y');
    theta = np.linalg.lstsq(poly_basis(x, N=N_params), y, rcond=None)[0]
    ax.plot(x_plot, np.dot(poly_basis(x_plot, N=N_params), theta), 'k');
widgets.interact(update_plot, N_params=IntSlider_nice(min=1, max=20));

# Complejidad, sobreajuste y generalización

Un modelo con más parámetros es más flexible pero también más complejo

> Complejidad: grados de libertad de un modelo

Un exceso de flexibilidad no es bueno. Podría ocurrir que el modelo **se ajuste al ruido** 

> Sobreajuste: Ocurre cuando el modelo "memoriza" los datos

Cuando esto ocurre el modelo pierde capacidad de generalización

> Generalización: Capacidad de predecir adecuadamente datos no usados en el ajuste

Tres maneras de evitar el sobreajuste y mejorar la capacidad de generalización

- Usar modelos de baja complejidad 
- Escoger la complejidad mediante pruebas de validación 
- Usar **regularización**

# Validación

Consiste en dividir el conjunto de datos en 3 subconjuntos
1. Entrenamiento: Datos que se ocupan para **ajustar el modelo**
1. Validación: Datos que se ocupan para **calibrar el modelo**
1. Prueba: Datos que se ocupan para comparar distintos modelos y estimar el error de generalización

Los datos de validación y prueba están "escondidos" para el modelo

> Ajustamos nuestro modelo minimizando el error de entrenamiento

> Seleccionamos la cantidad de parámetros minimizando el error de validación

Un modelo sobreajustado tiene buen desempeño en entrenamiento y malo en validación

Tipicamente se hacen **particiones aleatorias** del conjunto original

Lo más importante es que las particiones sean **representativas**

Retomaremos este tema más adelante...

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
theta = [10, -2, -0.3, 0.1]
x = np.linspace(-5, 6, num=21); 
X = poly_basis(x, len(theta))
y = np.dot(X, theta)

rseed, sigma = 0, 1.
np.random.seed(rseed);
Y = y + sigma*np.random.randn(len(x))
P = np.random.permutation(len(x))
train_idx, valid_idx = P[:len(x)//2], P[len(x)//2:]

def update_plot(ax, N):
    ax.cla();
    Phi = poly_basis(x, N)
    theta_hat = np.linalg.lstsq(Phi[train_idx, :], Y[train_idx], rcond=None)[0]
    ax.plot(x, y, 'b-', linewidth=2, label='Implícito', alpha=0.5, zorder=-100)
    ax.scatter(x[train_idx], Y[train_idx], c='b', s=50, label='Entrenamiento')
    ax.scatter(x[valid_idx], Y[valid_idx], c='g', s=50, label='Validación')
    ax.vlines(x[train_idx], np.dot(Phi[train_idx, :], theta_hat), Y[train_idx], 'b')  
    ax.vlines(x[valid_idx], np.dot(Phi[valid_idx, :], theta_hat), Y[valid_idx], 'g')     
    x_plot = np.linspace(-5, 6, num=100);
    ax.plot(x_plot, np.dot(poly_basis(x_plot, N), theta_hat), 'k-', linewidth=2, label='Modelo')
    ax.set_ylim([-5, 15]); plt.legend()
    
interact(update_plot, ax=widgets.fixed(ax), N=IntSlider_nice(min=1, max=11));

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
N_values = np.arange(1, 11)
mse = np.zeros(shape=(len(N_values), 2))
for i, N in enumerate(N_values):
    Phi = poly_basis(x, N)
    theta_hat = np.linalg.lstsq(Phi[train_idx, :], Y[train_idx], rcond=None)[0]
    mse[i, 0] = np.mean(np.power(Y[train_idx] - np.dot(Phi[train_idx, :], theta_hat), 2))
    mse[i, 1] = np.mean(np.power(Y[valid_idx] - np.dot(Phi[valid_idx, :], theta_hat), 2))
ax.plot(N_values, mse[:, 1], label='Validación')
ax.plot(N_values, mse[:, 0], label='Entrenamiento')
idx_best = np.argmin(mse[:, 1])
ax.scatter(N_values[idx_best], mse[idx_best, 1])
plt.legend()
ax.set_ylim([1e-4, 1e+4])
ax.set_yscale('log')

# Regularización

Consiste en agregar una penalización adicional al problema 

El ejemplo clásico es pedir que la solución tenga norma mínima

$$
\min_x \|Ax-b\|_2^2 + \lambda \|x\|_2^2
$$

En este caso la solución es

$$
\hat x = (A^T A + \lambda I)^{-1} A^T b
$$

que se conoce como **ridge regression** o **regularización de Tikhonov**

$\lambda$ es un hiper-parámetro del modelo y debe ser escogido por el usuario (usando validación)

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
x = np.linspace(-5, 5, num=20); 
x_plot = np.linspace(-5, 5, num=200); 
model = np.sin(x)*x + 0.1*x**2

def update(N, lamb, sigma, rseed):
    np.random.seed(rseed); 
    y = model + sigma*np.random.randn(len(x))
    y = (y - np.mean(y))/np.sqrt(np.sum(y**2))
    X = poly_basis(x, N); X_plot = poly_basis(x_plot, N)
    Lamb = lamb*np.identity(N); Lamb[0, 0] = 0
    theta = np.linalg.solve(np.dot(X.T, X) + Lamb, np.dot(X.T, y))
    ax.cla(); 
    ax.plot(x_plot , np.dot(X_plot, theta), 'k-', linewidth=2, label='Modelo')
    ax.scatter(x, y, c='b', s=30, label='Datos', zorder=100); plt.legend()

    
interact(update, rseed=IntSlider_nice(min=1, max=10), 
         N=SelSlider_nice(options=[1, 2, 3, 5, 7, 10, 15, 17, 20]), 
         sigma=SelSlider_nice(options=[0.1, 1, 2, 5]),
         lamb=SelSlider_nice(options=[0.0, 0.1, 1., 10., 1e+2, 1e+3, 1e+4, 1e+5, 1e+6]));


Opciones de más alto nivel para hacer regresión lineal

- `scipy.stats.linregress`
- [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [ ]:
from sklearn.linear_model import Ridge
#from sklearn.preprocessing import PolynomialFeatures
#from sklearn.pipeline import make_pipeline

plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
x = np.linspace(-5, 5, num=20); 
x_plot = np.linspace(-5, 5, num=200); 
model = np.sin(x)*x + 0.1*x**2

def update(N, lamb, sigma, rseed):
    np.random.seed(rseed); 
    y = model + sigma*np.random.randn(len(x))
    X = poly_basis(x, N); X_plot = poly_basis(x_plot, N)
    #regressor = make_pipeline(PolynomialFeatures(N-1), 
    #                          Ridge(normalize=True, fit_intercept=True, alpha=lamb))
    regressor = Ridge(normalize=True, fit_intercept=True, alpha=lamb)
    regressor.fit(X, y)
    ax.cla(); 
    ax.plot(x_plot , regressor.predict(X_plot), 'k-', linewidth=2, label='Modelo')
    ax.scatter(x, y, c='b', s=30, label='Datos', zorder=100); plt.legend()

    
interact(update, rseed=IntSlider_nice(min=1, max=10), 
         N=SelSlider_nice(options=[1, 2, 3, 5, 7, 10, 15, 17, 20]), 
         sigma=SelSlider_nice(options=[0.1, 1, 2, 5]),
         lamb=SelSlider_nice(options=[0.0, 1e-4, 1e-3, 1e-2, 1e-1, 1., 10.]));

## Actividad práctica: Predicción de calidad de aire

Considere el siguiente data-set
https://archive.ics.uci.edu/ml/datasets/Air+Quality

- Importe y "parsee" el dataset usando pandas
- Entrene un predictor para la concentración de CO y otro para el Benzeno (C6H6)
- Use la primera semana de datos para ajustar y la siguiente semana para probar
- Observe los datos ¿Qué tipo de modelo es apropiado en este caso? 
- Encuentre la cantidad de parámetros que minimiza el error de validación

Ojo: Un valor de "-200" es un *missing value*

Documentación pandas timestamp: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html

In [ ]:
%%bash
wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
unzip AirQualityUCI.zip

In [ ]:
import pandas as pd
df = pd.read_csv("AirQualityUCI.csv", sep=';', decimal=',')
df = df[df.columns[:-2]] 
df.dropna(inplace=True)
df["Date_time"] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%d/%m/%Y %H.%M.%S")
df.drop(["Date", "Time"], axis=1, inplace=True)
df.head(n=5)

In [ ]:
#df.loc[(df["Date_time"] < pd.to_datetime("03/24/2004")) & (df["CO(GT)"] > -200)][["Date_time", "CO(GT)"]].plot(x=0, y=1)
time, data = df.loc[(df["Date_time"] < pd.to_datetime("03/24/2004 18:00:00")) & (df["CO(GT)"] > -200)][["Date_time", "CO(GT)"]].values.T
data = data.astype('float32')
time_float = np.array([t.timestamp()/(24*3600) - time[0].timestamp()/(24*3600) for t in time])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mask = time_float <= 7
ax.plot(time_float[mask], data[mask])

def build_phi(time, freqs=[1]):
    phi_cos = np.stack([np.cos(2*np.pi*time*f) for f in [0]+freqs])
    phi_sin = np.stack([np.sin(2*np.pi*time*f) for f in freqs])
    return np.concatenate((phi_cos, phi_sin), axis=0).T

X_lstsq = build_phi(time_float[mask])
theta, MSE, rank, singval = np.linalg.lstsq(X_lstsq, data[mask], rcond=None)
ax.plot(time_float[mask], np.dot(X_lstsq, theta))
mask = time_float > 7
ax.plot(time_float[mask], data[mask])
X_lstsq = build_phi(time_float[mask])
mse = np.mean(np.power(np.dot(X_lstsq, theta) - data[mask], 2))
display("Mean Square Error:"+str(mse))
ax.plot(time_float[mask], np.dot(X_lstsq, theta));

In [ ]:
%%bash
rm AirQualityUCI*

FUTURO: outliers